In [1]:
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

from Qgps import GPS

In [2]:
data_path = "calsource_orientation.dat"
base_antenna_position = np.array([0, 0, 0])
distance_between_antennas = 1
distance_calsource = 0.5
observation_date = np.array([dt.datetime(2024, 3, 22, 12, 47, 49), dt.datetime(2024, 3, 22, 12, 48, 49)])

In [3]:
ex = np.array([1, 0, 0])
ey = np.array([0, 1, 0])
print(np.dot(ex, ey))

0


In [4]:
gps = GPS(data_path, base_antenna_position, distance_between_antennas, distance_calsource, observation_date)

0 (3, 480)
0 bis (480,)
1 (3,)
2 bis [array([0.33675344, 0.33674456, 0.33675011, 0.33674572, 0.3367486 ,
        0.33674858, 0.33674193, 0.33676455, 0.33675459, 0.33675338,
        0.33675972, 0.33675019, 0.33676619, 0.33677006, 0.33674693,
        0.33675661, 0.33675958, 0.33675741, 0.33675936, 0.33675026,
        0.33675535, 0.33675662, 0.3367467 , 0.33675287, 0.33674856,
        0.33674567, 0.33674953, 0.33674523, 0.33674572, 0.33674696,
        0.33674082, 0.33675976, 0.33675085, 0.33675044, 0.3367538 ,
        0.33675219, 0.33675863, 0.33676465, 0.33675711, 0.33676647,
        0.33675723, 0.33675842, 0.33675981, 0.33675999, 0.3367496 ,
        0.33676035, 0.33674383, 0.3367628 , 0.33676536, 0.33675359,
        0.33676431, 0.33676327, 0.33675996, 0.3367586 , 0.33675643,
        0.33675538, 0.33676176, 0.336755  , 0.33675099, 0.3367503 ,
        0.33675791, 0.33674621, 0.33675258, 0.33676234, 0.33674294,
        0.33674335, 0.33674609, 0.33674204, 0.33674021, 0.33675588,
        0.3

/home/tlaclave/qubic/qubic/scripts/GPS system/Qgps.py:301: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  rotation_matrix = np.array([[np.cos(theta), -np.sin(theta), 0],


ValueError: setting an array element with a sequence.

In [5]:
gps.timestamp

3

In [7]:
gps.datetime

array([datetime.datetime(2024, 3, 22, 9, 47, 49),
       datetime.datetime(2024, 3, 22, 9, 47, 49, 125000),
       datetime.datetime(2024, 3, 22, 9, 47, 49, 250000), ...,
       datetime.datetime(2024, 3, 22, 16, 13, 49, 250000),
       datetime.datetime(2024, 3, 22, 16, 13, 49, 375000),
       datetime.datetime(2024, 3, 22, 16, 13, 49, 500000)], dtype=object)

In [6]:
gps.position_antenna1

array([[0.03732225, 0.13927341, 0.42760998],
       [0.03732869, 0.14064032, 0.42830998],
       [0.03569961, 0.13992352, 0.43080998],
       ...,
       [0.02592055, 0.14030546, 0.42205859],
       [0.02548302, 0.13898708, 0.42181049],
       [0.02390986, 0.13962286, 0.42262084]])

In [8]:
gps.position_antenna2

array([[ 0.9927, -0.1561,  0.4248],
       [ 0.9925, -0.1554,  0.4255],
       [ 0.991 , -0.1557,  0.428 ],
       ...,
       [ 0.9806, -0.1573,  0.4178],
       [ 0.9801, -0.1588,  0.4164],
       [ 0.9784, -0.1586,  0.4192]])

In [9]:
gps.position_calsource

array([[ 0.51501112, -0.00841329,  0.42620499],
       [ 0.51491434, -0.00737984,  0.42690499],
       [ 0.5133498 , -0.00788824,  0.42940499],
       ...,
       [ 0.50326027, -0.00849727,  0.4199293 ],
       [ 0.50279151, -0.00990646,  0.41910525],
       [ 0.50115493, -0.00948857,  0.42091042]])